<a href="https://colab.research.google.com/github/Apoorvkhanna2/Insuranceit/blob/main/Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install required dependencies (run this cell first)
!pip install gradio scikit-learn pandas numpy

import gradio as gr
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# ------------------------------------------------------------------
# Define default feature columns (if model isn't trained yet)
default_feature_columns = [
    "Age", "Is_Senior", "Marital_Status", "Married_Premium_Discount",
    "Prior_Insurance", "Prior_Insurance_Premium_Adjustment", "Claims_Frequency",
    "Claims_Severity", "Claims_Adjustment", "Policy_Type", "Policy_Adjustment",
    "Premium_Amount", "Safe_Driver_Discount", "Multi_Policy_Discount", "Bundling_Discount",
    "Total_Discounts", "Source_of_Lead", "Time_Since_First_Contact", "Website_Visits",
    "Inquiries", "Quotes_Requested", "Time_to_Conversion", "Credit_Score",
    "Premium_Adjustment_Credit", "Region", "Premium_Adjustment_Region"
]

# Define which features are categorical
categorical_features = ["Marital_Status", "Prior_Insurance", "Claims_Severity", "Policy_Type", "Source_of_Lead", "Region"]

# Global variables to store trained model, label encoders, and feature columns
stored_model, stored_label_encoders, stored_feature_columns = None, None, None

# ------------------------------------------------------------------
# Function to load dataset from the uploaded CSV file
def load_dataset(file):
    df = pd.read_csv(file.name)
    return df

# Function to train the model from the dataframe and return model, encoders, and feature columns
def train_model(df):
    # Create label encoders for categorical features
    label_encoders = {feature: LabelEncoder() for feature in categorical_features}

    # Encode categorical features
    for feature in categorical_features:
        df[feature] = label_encoders[feature].fit_transform(df[feature])

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier

    # Assuming 'Conversion_Status' is the target column
    X = df.drop(columns=["Conversion_Status"])
    y = df["Conversion_Status"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Save the trained model to Google Drive
    model_path = "/content/drive/My Drive/insurance_model.pkl"
    joblib.dump(model, model_path)

    # Return the model, label encoders, and the list of feature columns
    return model, label_encoders, list(X.columns)

# Function to preprocess user input before prediction
def preprocess_input(*args, label_encoders, feature_columns):
    df_input = pd.DataFrame([args], columns=feature_columns)
    for feature in categorical_features:
        df_input[feature] = label_encoders[feature].transform(df_input[feature])
    return df_input.values.reshape(1, -1)

# Prediction function using the trained model
def predict_insurance(*args):
    if stored_model is None:
        return "Please train the model first by uploading a CSV file."
    processed_features = preprocess_input(*args, label_encoders=stored_label_encoders, feature_columns=stored_feature_columns)
    prediction = stored_model.predict(processed_features)[0]
    return prediction

# ------------------------------------------------------------------
# Interface function to handle CSV upload and model training
def handle_csv_upload(file):
    global stored_model, stored_label_encoders, stored_feature_columns
    df = load_dataset(file)
    stored_model, stored_label_encoders, stored_feature_columns = train_model(df)
    return "Model Trained Successfully!"

# Create the training interface (upload CSV & train model)
train_interface = gr.Interface(fn=handle_csv_upload, inputs=gr.File(label="Upload CSV File"), outputs="text", live=True)

# For the prediction interface, use the stored_feature_columns if available; otherwise, use the default list.
used_feature_columns = stored_feature_columns if stored_feature_columns is not None else default_feature_columns

# Create input components for prediction based on the feature columns
pred_inputs = [
    gr.Number(label=col) if col not in categorical_features else gr.Textbox(label=col)
    for col in used_feature_columns
]

# Create the prediction interface
prediction_interface = gr.Interface(
    fn=predict_insurance,
    inputs=pred_inputs,
    outputs=gr.Textbox(label="Predicted Insurance Outcome"),
    title="Insurance Prediction"
)

# ------------------------------------------------------------------
# Launch both interfaces in a tabbed layout
tab_interface = gr.TabbedInterface([train_interface, prediction_interface], ["Train Model", "Make Predictions"])
tab_interface.launch(share=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67a8aed215063e1a57.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
